데이터 전처리

In [4]:
import pandas as pd

region_name = '부산'
# Step 1: Load the initial dataset
file_path = f"C:/Users/USER/Desktop/python/data/{region_name}_빈집_20년이하.CSV"  # Replace with your initial file path
data = pd.read_csv(file_path, encoding='euc-kr')

# Add '20년이하' column
data['20년이하'] = data['전체 빈집수']

# Step 2: Define the files and corresponding new columns
file_column_mapping = {
    f"C:/Users/USER/Desktop/python/data/{region_name}_빈집_20년이상30년이하.CSV": '20년이상30년미만',
    f"C:/Users/USER/Desktop/python/data/{region_name}_빈집_30년이상40년이하.CSV": '30년이상40년미만',
    f"C:/Users/USER/Desktop/python/data/{region_name}_빈집_40년이상.CSV": '40년이상'
}

# Step 3: Add columns for new age ranges
for column in file_column_mapping.values():
    data[column] = 0

# Step 4: Process each file and update the data
for file, column in file_column_mapping.items():
    new_data = pd.read_csv(file, encoding='euc-kr')
    
    # Clean numeric data
    new_data['전체 빈집수'] = new_data['전체 빈집수'].replace(',', '', regex=True).astype(float)
    numeric_columns = data.columns[data.columns.get_loc('전체 빈집수'):data.columns.get_loc('기타')+1]
    new_data[numeric_columns] = new_data[numeric_columns].apply(
        lambda col: col.replace(',', '', regex=True).astype(float) if col.dtype == 'object' else col
    )
    
    # Update or append rows based on '지역' match
    for _, row in new_data.iterrows():
        region = row['지역']
        matching_row = data[data['지역'] == region]
        if not matching_row.empty:
            idx = matching_row.index[0]
            data.loc[idx, numeric_columns] += row[numeric_columns]
            data.loc[idx, column] = row['전체 빈집수']
        else:
            new_row = pd.DataFrame([row], columns=new_data.columns)
            new_row[column] = row['전체 빈집수']
            data = pd.concat([data, new_row], ignore_index=True)

# Step 5: Save the result to a CSV file
output_file_path = f"{region_name}빈집_최종.csv"
data.insert(0, '지역분류',region_name)
data.to_csv(output_file_path, index=False, encoding='euc-kr')

print(f"CSV file saved at: {output_file_path}")

CSV file saved at: 부산빈집_최종.csv
